In [1889]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
from fake_headers import Headers

header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

In [1890]:
headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.zillow.com/ok/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22OK%22%2C%22mapBounds%22%3A%7B%22west%22%3A-104.90186123437499%2C%22east%22%3A-92.53125576562499%2C%22south%22%3A32.436077585580584%2C%22north%22%3A38.11770469384586%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A45%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22built%22%3A%7B%22min%22%3A0%2C%22max%22%3A2010%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A7%7D',
    'accept-language': 'en-US,en;q=0.9',
}
list1112221=['new','fresh%20paint','quartz','remodeled','renovated','shiplap','soft%20close','staging','stainless','turn%20key','updated']

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.88620504589845%2C%22east%22%3A-117.93726095410157%2C%22south%22%3A33.54554919445917%2C%22north%22%3A34.49481997322805%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sortSelection%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22isAllHomes%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D&wants=\{%22cat1%22:\[%22listResults%22,%22mapResults%22\],%22cat2%22:\[%22total%22\]\}&requestId=3', headers=headers)


In [1891]:
#kword = 'turn%20key'
kword = 'updated'
state='WA'
with requests.session() as s:
    page = 1
    end_page = 20
    url = ''
    url_list = []
    
    while page <= end_page:
        
        url = 'https://www.zillow.com/wa/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A'+str(page)+'%7D%2C%22usersSearchTerm%22%3A%22WA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-127.067579734375%2C%22east%22%3A-114.696974265625%2C%22south%22%3A44.88671077042072%2C%22north%22%3A49.61043400158961%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A59%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22att%22%3A%7B%22value%22%3A%22'+kword+'%22%7D%2C%22built%22%3A%7B%22max%22%3A2010%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A7%7D'
        url_list.append(url)
        page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        request = s.get(url, headers=headers)
        request_list.append(request)
    
soup = ''
soup_list = []

for request in request_list:
    soup = BeautifulSoup(request.content, 'lxml')
    soup_list.append(soup)

In [1892]:
df_list = []
for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        address = soup.find_all (class_= 'list-card-addr')
        price = list(soup.find_all (class_='list-card-price'))
        beds = list(soup.find_all("ul", class_="list-card-details"))
        details = soup.find_all ('div', {'class': 'list-card-details'})
        home_type = soup.find_all ('div', {'class': 'list-card-footer'})
        last_updated = soup.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        link = soup.find_all (class_= 'list-card-link')

        df['prices'] = price
        df['address'] = address
        df['beds'] = beds
        
    urls = []
    
    for link in soup.find_all("article"):
          try:
               href = link.find('a',class_="list-card-link")['href']
          except Exception as e:
               href=''
		#raise e 

          urls.append(href)
    if '' in urls:
          urls.remove('')
    df['links'] = urls
    df['links'] = df['links'].astype('str')
    df_list.append(df)

In [1893]:
df = pd.concat(df_list).reset_index().drop(columns='index')

In [1894]:
df['address'] = df['address'].astype('str')
df['beds'] = df['beds'].astype('str')

In [1895]:
#remove html tags
#df['prices'] = df['prices'].replace('\[', '', regex=True)
df.loc[:,'address'] = df.loc[:,'address'].replace('<address class="list-card-addr">', '', regex=True)
#df['prices'] = df['prices'].replace('\]', '', regex=True)
df.loc[:,'address'] = df.loc[:,'address'].replace('</address>', '', regex=True)
#df['prices'] = df['prices'].str.replace(r'\D', '')

#filter unwanted property types
df = df[~df['beds'].str.contains("Land for sale")]

#remove html tags from beds column
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!-- -->Foreclosure</abbr>', '- Foreclosure', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('</li></ul>', '', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('--', '0', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('Multi-family', 'Multifamily', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace(' for sale', '', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Auction</abbr>', '- Auction', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Pending</abbr>', '- Pending', regex=True)

#split beds column into beds, bath and sq_feet
#df[['beds','baths','sq_feet']] = df.beds.str.split(expand=True)

In [1896]:
df['bed'] = df.beds.apply(
    lambda x: pd.Series(str(x).split('-')[0]))


In [1897]:
df['type'] = df.beds.apply(
    lambda x: pd.Series(str(x).split('-')[1]))

In [1898]:
df.head()

,prices,address,beds,links,bed,type
0,"[$565,600]","7510 NW 11th Ave, Vancouver, WA 98665","3 3 2,322 - House",https://www.zillow.com/homedetails/7510-NW-11t...,"3 3 2,322",House
1,"[$689,200]","117 W 37th St, Vancouver, WA 98660","4 3 2,548 - House",https://www.zillow.com/homedetails/117-W-37th-...,"4 3 2,548",House
2,"[$634,900]","8901 NW 12th Ave, Vancouver, WA 98665","4 4 2,900 - House",https://www.zillow.com/homedetails/8901-NW-12t...,"4 4 2,900",House
3,"[$399,000]","19424 Military Rd S, Seatac, WA 98188","2 2 1,110 -<abbr class=""list-card-label""> <!0...",https://www.zillow.com/homedetails/19424-Milit...,"2 2 1,110","<abbr class=""list"
4,"[$475,000]","4700 NW Lavina St, Vancouver, WA 98663","3 2 1,389 -<abbr class=""list-card-label""> <!0...",https://www.zillow.com/homedetails/4700-NW-Lav...,"3 2 1,389","<abbr class=""list"


In [1899]:
df[['bed', 'baths', 'sq_feet']] = df.bed.str.split(expand=True)

In [1900]:
df['prices']=df.prices.apply(
    lambda x: pd.Series(str(x).split('>')[1].split('<')[0]))

In [1901]:
urls=df['links']

In [1902]:
urls1=urls.tolist()

In [1903]:
import json
agent=[]
contact=[]
agentComp=[]
des=[]
for i in urls:
    headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': i,
    'accept-language': 'en-US,en;q=0.9',
    }
    response=requests.get(i,headers=headers)
    soup=BeautifulSoup(response.content,'lxml')
    data = json.loads(soup.find_all('script', type='application/json')[3].string)
    data1=json.loads(data['apiCache'])
    l1=list(data1.keys())
    a=(str(l1[1]))
    data2=data1[a]['property']['attributionInfo']
    agent.append(data2['infoString4'])
    contact.append(data2['infoString5'])
    agentComp.append(data2['infoString6'])
    des.append(data1[a]['property']['description'])



In [1904]:
df['Listing Agent Name']=agent
df['Listing Agent Contact']=contact
df['Listing Agent Company']=agentComp
df['Description']=des

In [1905]:
df['Key Word']=kword

In [1906]:
df=df.drop(['beds'],axis=1)

In [1907]:
df.to_csv(state+'-'+kword+'.csv')